In [1]:
#import the libraries

import os
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training, AdaLoraConfig

from transformers import TrainingArguments

In [3]:
import numpy as np
import torch
#from datasets import load_dataset, load_metric
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

#import the dataset
from datasets import load_dataset

raw_datasets  = load_dataset("glue", 'sst2')

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"
# Load the configuration of the pre-trained model
config = AutoConfig.from_pretrained(model_name)

config.hidden_dropout_prob=0.0
config.attention_probs_dropout_prob=0.00
# Load the tokenizer for the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding

col_to_delete = ['idx', 'sentence']

def preprocessing_function(examples):
    return tokenizer(examples['sentence'], truncation=True, max_length=128)

tokenized_datasets = raw_datasets.map(preprocessing_function, batched=True, remove_columns=col_to_delete)

tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [7]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random

In [8]:
from transformers import RobertaForSequenceClassification
# Load a pre-trained Roberta model for sequence classification
model = RobertaForSequenceClassification.from_pretrained(
    model_name,

)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

# Define LoRA configuration parameters
lora_alpha = 8 
lora_dropout = 0.1 
lora_rank = 4 
# Initialize the LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        
        "query",
        "key",
        "value",
        "dense",
    ]
)
# Apply the LoRA configuration to the model
model = get_peft_model(model, peft_config)

In [10]:
model = model.to('cuda')

In [11]:
import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)
    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [12]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='lorabase',
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.00,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/s/sami20/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Train the model
trainer.train()

/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,0.704200,0.692571,0.560854,0.536657,0.488563,0.542431
200,0.631800,0.378517,0.870311,0.849762,0.845645,0.847477
300,0.328600,0.226086,0.918539,0.918572,0.918554,0.918578
400,0.259600,0.228207,0.919261,0.918235,0.918475,0.918578
500,0.256100,0.201543,0.921992,0.921992,0.921992,0.922018
600,0.243000,0.207405,0.925451,0.925413,0.925430,0.925459
700,0.227400,0.207120,0.927943,0.927581,0.927702,0.927752
800,0.217100,0.205153,0.928636,0.927370,0.927649,0.927752
900,0.217400,0.203315,0.927715,0.927749,0.927731,0.927752
1000,0.200400,0.215396,0.927477,0.924865,0.925277,0.925459


/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/s/sami20/miniconda3/lib/python3.12

TrainOutput(global_step=1404, training_loss=0.29052753721037483, metrics={'train_runtime': 372.4283, 'train_samples_per_second': 361.675, 'train_steps_per_second': 3.77, 'total_flos': 3321706576487628.0, 'train_loss': 0.29052753721037483, 'epoch': 2.0})